In [ ]:
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

In [ ]:
# function to read raw text file
def read_text(filename):
        # open the file
        file = open(filename, mode='rt', encoding='utf-8')

        # read all text
        text = file.read()
        file.close()
        return text

In [ ]:
# split a text into sentences
def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t') for i in sents]
    return sents

In [ ]:
data = read_text("/content/deu.txt")
eng_pes = to_lines(data)
eng_pes = array(eng_pes)

In [ ]:
eng_pes

array([['Go.', 'Geh.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)'],
       ['Hi.', 'Hallo!',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)'],
       ['Hi.', 'Grüß Gott!',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)'],
       ...,
       ["If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.",
        'Wenn jemand Fremdes dir sagt, dass du dich wie ein Muttersprachler anhörst, bedeutet das wahrscheinlich: Er hat etwas an deinem Sprechen bemerkt, dass dich als Nicht-Muttersprachler verraten hat. Mit anderen Worten: Du hörst dich nicht wirklich wie ein Muttersprachler an.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #3807493 (Tickler)'],
       ["If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.",
        'Wenn jemand, der nicht weiß, woher man kommt, sagt, man erwecke doch den Eindruck, Muttersprachler zu sein, so hat man Grund zu der Annahme, dass ihm an der Sprache irgendetwas aufgefallen ist, woran er erkannt hat, dass man eben keiner ist\xa0– dass man diesen Eindruck mit anderen Worten eigentlich nicht erweckt.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #8836704 (Pfirsichbaeumchen)'],
       ['Doubtless there exists in this world precisely the right woman for any given man to marry and vice versa; but when you consider that a human being has the opportunity of being acquainted with only a few hundred people, and out of the few hundred that there are but a dozen or less whom he knows intimately, and out of the dozen, one or two friends at most, it will easily be seen, when we remember the number of millions who inhabit this world, that probably, since the earth was created, the right man has never yet met the right woman.',
        'Ohne Zweifel findet sich auf dieser Welt zu jedem Mann genau die richtige Ehefrau und umgekehrt; wenn man jedoch in Betracht zieht, dass ein Mensch nur Gelegenheit hat, mit ein paar hundert anderen bekannt zu sein, von denen ihm nur ein Dutzend oder weniger nahesteht, darunter höchstens ein oder zwei Freunde, dann erahnt man eingedenk der Millionen Einwohner dieser Welt\xa0leicht, dass seit Erschaffung ebenderselben wohl noch nie der richtige Mann der richtigen Frau begegnet ist.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #7697649 (RM) & #7729416 (Pfirsichbaeumchen)']],
      dtype='<U537')

In [ ]:
eng_pes = eng_pes[:50000,:]

In [ ]:
# Remove punctuation
eng_pes[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in eng_pes[:,0]]
eng_pes[:,1] = [s.translate(str.maketrans('', '', string.punctuation)) for s in eng_pes[:,1]]

eng_pes


array([['Go', 'Geh',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)'],
       ['Hi', 'Hallo',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)'],
       ['Hi', 'Grüß Gott',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)'],
       ...,
       ['Im giving up smoking', 'Ich höre mit dem Rauchen auf',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #256952 (minshirui) & #407184 (MUIRIEL)'],
       ['Im glad I was nearby', 'Ich bin froh dass ich in der Nähe war',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2547219 (CK) & #3448316 (Pfirsichbaeumchen)'],
       ['Im glad Tom has gone', 'Ich bin froh dass Tom weg ist',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2547217 (CK) & #5299642 (Pfirsichbaeumchen)']],
      dtype='<U537')

In [ ]:
# convert text to lowercase
for i in range(len(eng_pes)):
    eng_pes[i,0] = eng_pes[i,0].lower()
    eng_pes[i,1] = eng_pes[i,1].lower()

In [ ]:
# empty lists
eng_l = []
pes_l = []

# populate the lists with sentence lengths
for i in eng_pes[:,0]:
      eng_l.append(len(i.split()))

for i in eng_pes[:,1]:
      pes_l.append(len(i.split()))

length_df = pd.DataFrame({'eng':eng_l, 'pes':pes_l})

length_df.hist(bins = 30)
plt.show()

In [ ]:
# function to build a tokenizer
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# prepare english tokenizer
eng_tokenizer = tokenization(eng_pes[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1

eng_length = 8
print('English Vocabulary Size: %d' % eng_vocab_size)

English Vocabulary Size: 6256

In [ ]:
# prepare Persian tokenizer
pes_tokenizer = tokenization(eng_pes[:, 1])
pes_vocab_size = len(pes_tokenizer.word_index) + 1

pes_length = 8
print('German Vocabulary Size: %d' % pes_vocab_size)

German Vocabulary Size: 10329

In [ ]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

# Model

In [ ]:
from sklearn.model_selection import train_test_split

# split data into train and test set
train, test = train_test_split(eng_pes, test_size=0.2, random_state = 12)

In [ ]:
# prepare training data
trainX = encode_sequences(pes_tokenizer, pes_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])

# prepare validation data
testX = encode_sequences(pes_tokenizer, pes_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import MultiHeadAttention

In [ ]:
#Proposed Model to just add Bidirectional LSTM
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras import optimizers
def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(Bidirectional(LSTM(units, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(units)))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))  # Regular LSTM in decoder
    model.add(Dense(out_vocab, activation='softmax'))

    rms = optimizers.RMSprop(lr=0.001)
    model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

    return model

In [ ]:
#multi-head_self attention with bi-directional lstm
from tensorflow.keras.layers import MultiHeadAttention, RepeatVector, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras import optimizers

def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    input_layer = Input(shape=(in_timesteps,))
    embedding_layer = Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True)(input_layer)
    lstm_layer = Bidirectional(LSTM(units, return_sequences=True))(embedding_layer)
    attention = MultiHeadAttention(num_heads=2, key_dim=units)(lstm_layer, lstm_layer)
    dropout = Dropout(0.5)(attention)
    lstm_layer_2 = Bidirectional(LSTM(units))(dropout)
    repeat_vector = RepeatVector(out_timesteps)(lstm_layer_2)
    lstm_layer_3 = LSTM(units, return_sequences=True)(repeat_vector)
    output_layer = Dense(out_vocab, activation='softmax')(lstm_layer_3)

    model = Model(inputs=input_layer, outputs=output_layer)
    rms = optimizers.RMSprop(lr=0.001)
    model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

    return model


In [ ]:
#Kaggle Proposed Model
# build NMT model
def define_model(in_vocab,out_vocab, in_timesteps,out_timesteps,units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dense(out_vocab, activation='softmax'))
    rms = optimizers.RMSprop(lr=0.001)
    model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')
    return model


In [ ]:
# model compilation
model = define_model(pes_vocab_size, eng_vocab_size, pes_length, eng_length, 512)

In [ ]:
filename = 'Proposed_model_5'

checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=30, batch_size=512, validation_split = 0.2,callbacks=[checkpoint],
                    verbose=1)


Epoch 1/30
63/63 [==============================] - ETA: 0s - loss: 3.8573
Epoch 1: val_loss improved from inf to 3.09141, saving model to Proposed_model_5
63/63 [==============================] - 82s 902ms/step - loss: 3.8573 - val_loss: 3.0914
Epoch 2/30
63/63 [==============================] - ETA: 0s - loss: 2.9161
Epoch 2: val_loss improved from 3.09141 to 2.88892, saving model to Proposed_model_5
63/63 [==============================] - 51s 818ms/step - loss: 2.9161 - val_loss: 2.8889
Epoch 3/30
63/63 [==============================] - ETA: 0s - loss: 2.8231
Epoch 3: val_loss improved from 2.88892 to 2.85578, saving model to Proposed_model_5
63/63 [==============================] - 50s 804ms/step - loss: 2.8231 - val_loss: 2.8558
Epoch 4/30
63/63 [==============================] - ETA: 0s - loss: 2.7807
Epoch 4: val_loss improved from 2.85578 to 2.79740, saving model to Proposed_model_5
63/63 [==============================] - 50s 809ms/step - loss: 2.7807 - val_loss: 2.7974
Epoch 5/30
63/63 [==============================] - ETA: 0s - loss: 2.7493
Epoch 5: val_loss improved from 2.79740 to 2.75987, saving model to Proposed_model_5
63/63 [==============================] - 50s 808ms/step - loss: 2.7493 - val_loss: 2.7599
Epoch 6/30
63/63 [==============================] - ETA: 0s - loss: 2.7224
Epoch 6: val_loss improved from 2.75987 to 2.73565, saving model to Proposed_model_5
63/63 [==============================] - 49s 790ms/step - loss: 2.7224 - val_loss: 2.7357
Epoch 7/30
63/63 [==============================] - ETA: 0s - loss: 2.6880
Epoch 7: val_loss improved from 2.73565 to 2.68262, saving model to Proposed_model_5
63/63 [==============================] - 52s 842ms/step - loss: 2.6880 - val_loss: 2.6826
Epoch 8/30
63/63 [==============================] - ETA: 0s - loss: 2.6538
Epoch 8: val_loss improved from 2.68262 to 2.66822, saving model to Proposed_model_5
63/63 [==============================] - 52s 838ms/step - loss: 2.6538 - val_loss: 2.6682
Epoch 9/30
63/63 [==============================] - ETA: 0s - loss: 2.6278
Epoch 9: val_loss improved from 2.66822 to 2.64200, saving model to Proposed_model_5
63/63 [==============================] - 55s 877ms/step - loss: 2.6278 - val_loss: 2.6420
Epoch 10/30
63/63 [==============================] - ETA: 0s - loss: 2.6051
Epoch 10: val_loss did not improve from 2.64200
63/63 [==============================] - 9s 149ms/step - loss: 2.6051 - val_loss: 2.6480
Epoch 11/30
63/63 [==============================] - ETA: 0s - loss: 2.5882
Epoch 11: val_loss improved from 2.64200 to 2.62742, saving model to Proposed_model_5
63/63 [==============================] - 54s 871ms/step - loss: 2.5882 - val_loss: 2.6274
Epoch 12/30
63/63 [==============================] - ETA: 0s - loss: 2.5727
Epoch 12: val_loss improved from 2.62742 to 2.61444, saving model to Proposed_model_5
63/63 [==============================] - 54s 869ms/step - loss: 2.5727 - val_loss: 2.6144
Epoch 13/30
63/63 [==============================] - ETA: 0s - loss: 2.5488
Epoch 13: val_loss improved from 2.61444 to 2.59372, saving model to Proposed_model_5
63/63 [==============================] - 50s 802ms/step - loss: 2.5488 - val_loss: 2.5937
Epoch 14/30
63/63 [==============================] - ETA: 0s - loss: 2.5212
Epoch 14: val_loss improved from 2.59372 to 2.56922, saving model to Proposed_model_5
63/63 [==============================] - 55s 881ms/step - loss: 2.5212 - val_loss: 2.5692
Epoch 15/30
63/63 [==============================] - ETA: 0s - loss: 2.4936
Epoch 15: val_loss improved from 2.56922 to 2.51500, saving model to Proposed_model_5
63/63 [==============================] - 50s 799ms/step - loss: 2.4936 - val_loss: 2.5150
Epoch 16/30
63/63 [==============================] - ETA: 0s - loss: 2.4597
Epoch 16: val_loss improved from 2.51500 to 2.50523, saving model to Proposed_model_5
63/63 [==============================] - 50s 800ms/step - loss: 2.4597 - val_loss: 2.5052
Epoch 17/30
63/63 [==============================] - ETA: 0s - loss: 2.4649
Epoch 17: val_loss improved from 2.50523 to 2.46268, saving model to Proposed_model_5
63/63 [==============================] - 51s 822ms/step - loss: 2.4649 - val_loss: 2.4627
Epoch 18/30
63/63 [==============================] - ETA: 0s - loss: 2.3942
Epoch 18: val_loss improved from 2.46268 to 2.46084, saving model to Proposed_model_5
63/63 [==============================] - 52s 838ms/step - loss: 2.3942 - val_loss: 2.4608
Epoch 19/30
63/63 [==============================] - ETA: 0s - loss: 2.3614
Epoch 19: val_loss improved from 2.46084 to 2.41669, saving model to Proposed_model_5
63/63 [==============================] - 49s 793ms/step - loss: 2.3614 - val_loss: 2.4167
Epoch 20/30
63/63 [==============================] - ETA: 0s - loss: 2.3396
Epoch 20: val_loss improved from 2.41669 to 2.38914, saving model to Proposed_model_5
63/63 [==============================] - 50s 805ms/step - loss: 2.3396 - val_loss: 2.3891
Epoch 21/30
63/63 [==============================] - ETA: 0s - loss: 2.3130
Epoch 21: val_loss did not improve from 2.38914
63/63 [==============================] - 10s 151ms/step - loss: 2.3130 - val_loss: 2.4117
Epoch 22/30
63/63 [==============================] - ETA: 0s - loss: 2.2931
Epoch 22: val_loss improved from 2.38914 to 2.38278, saving model to Proposed_model_5
63/63 [==============================] - 49s 792ms/step - loss: 2.2931 - val_loss: 2.3828
Epoch 23/30
63/63 [==============================] - ETA: 0s - loss: 2.2725
Epoch 23: val_loss improved from 2.38278 to 2.34803, saving model to Proposed_model_5
63/63 [==============================] - 52s 829ms/step - loss: 2.2725 - val_loss: 2.3480
Epoch 24/30
63/63 [==============================] - ETA: 0s - loss: 2.2547
Epoch 24: val_loss improved from 2.34803 to 2.33404, saving model to Proposed_model_5
63/63 [==============================] - 54s 869ms/step - loss: 2.2547 - val_loss: 2.3340
Epoch 25/30
63/63 [==============================] - ETA: 0s - loss: 2.2362
Epoch 25: val_loss improved from 2.33404 to 2.32444, saving model to Proposed_model_5
63/63 [==============================] - 54s 873ms/step - loss: 2.2362 - val_loss: 2.3244
Epoch 26/30
63/63 [==============================] - ETA: 0s - loss: 2.2230
Epoch 26: val_loss improved from 2.32444 to 2.32287, saving model to Proposed_model_5
63/63 [==============================] - 55s 883ms/step - loss: 2.2230 - val_loss: 2.3229
Epoch 27/30
63/63 [==============================] - ETA: 0s - loss: 2.2049
Epoch 27: val_loss improved from 2.32287 to 2.28887, saving model to Proposed_model_5
63/63 [==============================] - 54s 873ms/step - loss: 2.2049 - val_loss: 2.2889
Epoch 28/30
63/63 [==============================] - ETA: 0s - loss: 2.1899
Epoch 28: val_loss did not improve from 2.28887
63/63 [==============================] - 10s 150ms/step - loss: 2.1899 - val_loss: 2.2898
Epoch 29/30
63/63 [==============================] - ETA: 0s - loss: 2.1798
Epoch 29: val_loss improved from 2.28887 to 2.26723, saving model to Proposed_model_5
63/63 [==============================] - 51s 817ms/step - loss: 2.1798 - val_loss: 2.2672
Epoch 30/30
63/63 [==============================] - ETA: 0s - loss: 2.1630
Epoch 30: val_loss improved from 2.26723 to 2.25981, saving model to Proposed_model_5
63/63 [==============================] - 53s 846ms/step - loss: 2.1630 - val_loss: 2.2598

In [ ]:
model = load_model('/content/kaggle_model')

In [ ]:
filename = 'kaggle_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=30, batch_size=512, validation_split = 0.2,callbacks=[checkpoint],
                    verbose=1)

Epoch 1/30
63/63 [==============================] - ETA: 0s - loss: 4.3083
Epoch 1: val_loss improved from inf to 3.11947, saving model to kaggle_model
63/63 [==============================] - 36s 423ms/step - loss: 4.3083 - val_loss: 3.1195
Epoch 2/30
63/63 [==============================] - ETA: 0s - loss: 2.9625
Epoch 2: val_loss improved from 3.11947 to 2.86550, saving model to kaggle_model
63/63 [==============================] - 16s 264ms/step - loss: 2.9625 - val_loss: 2.8655
Epoch 3/30
62/63 [============================>.] - ETA: 0s - loss: 2.8231
Epoch 3: val_loss improved from 2.86550 to 2.81203, saving model to kaggle_model
63/63 [==============================] - 16s 259ms/step - loss: 2.8230 - val_loss: 2.8120
Epoch 4/30
63/63 [==============================] - ETA: 0s - loss: 2.7767
Epoch 4: val_loss improved from 2.81203 to 2.77183, saving model to kaggle_model
63/63 [==============================] - 15s 243ms/step - loss: 2.7767 - val_loss: 2.7718
Epoch 5/30
63/63 [==============================] - ETA: 0s - loss: 2.7484
Epoch 5: val_loss improved from 2.77183 to 2.75870, saving model to kaggle_model
63/63 [==============================] - 15s 237ms/step - loss: 2.7484 - val_loss: 2.7587
Epoch 6/30
63/63 [==============================] - ETA: 0s - loss: 2.7311
Epoch 6: val_loss improved from 2.75870 to 2.74294, saving model to kaggle_model
63/63 [==============================] - 14s 221ms/step - loss: 2.7311 - val_loss: 2.7429
Epoch 7/30
63/63 [==============================] - ETA: 0s - loss: 2.7151
Epoch 7: val_loss improved from 2.74294 to 2.73751, saving model to kaggle_model
63/63 [==============================] - 17s 275ms/step - loss: 2.7151 - val_loss: 2.7375
Epoch 8/30
63/63 [==============================] - ETA: 0s - loss: 2.6974
Epoch 8: val_loss improved from 2.73751 to 2.72490, saving model to kaggle_model
63/63 [==============================] - 15s 240ms/step - loss: 2.6974 - val_loss: 2.7249
Epoch 9/30
63/63 [==============================] - ETA: 0s - loss: 2.6817
Epoch 9: val_loss improved from 2.72490 to 2.71372, saving model to kaggle_model
63/63 [==============================] - 15s 241ms/step - loss: 2.6817 - val_loss: 2.7137
Epoch 10/30
63/63 [==============================] - ETA: 0s - loss: 2.6527
Epoch 10: val_loss improved from 2.71372 to 2.67398, saving model to kaggle_model
63/63 [==============================] - 16s 253ms/step - loss: 2.6527 - val_loss: 2.6740
Epoch 11/30
63/63 [==============================] - ETA: 0s - loss: 2.6281
Epoch 11: val_loss improved from 2.67398 to 2.65918, saving model to kaggle_model
63/63 [==============================] - 14s 229ms/step - loss: 2.6281 - val_loss: 2.6592
Epoch 12/30
63/63 [==============================] - ETA: 0s - loss: 2.6072
Epoch 12: val_loss improved from 2.65918 to 2.63577, saving model to kaggle_model
63/63 [==============================] - 16s 261ms/step - loss: 2.6072 - val_loss: 2.6358
Epoch 13/30
63/63 [==============================] - ETA: 0s - loss: 2.5828
Epoch 13: val_loss improved from 2.63577 to 2.60439, saving model to kaggle_model
63/63 [==============================] - 15s 241ms/step - loss: 2.5828 - val_loss: 2.6044
Epoch 14/30
63/63 [==============================] - ETA: 0s - loss: 2.5602
Epoch 14: val_loss improved from 2.60439 to 2.59709, saving model to kaggle_model
63/63 [==============================] - 15s 242ms/step - loss: 2.5602 - val_loss: 2.5971
Epoch 15/30
63/63 [==============================] - ETA: 0s - loss: 2.5384
Epoch 15: val_loss improved from 2.59709 to 2.56795, saving model to kaggle_model
63/63 [==============================] - 14s 227ms/step - loss: 2.5384 - val_loss: 2.5679
Epoch 16/30
62/63 [============================>.] - ETA: 0s - loss: 2.5222
Epoch 16: val_loss improved from 2.56795 to 2.56513, saving model to kaggle_model
63/63 [==============================] - 15s 235ms/step - loss: 2.5218 - val_loss: 2.5651
Epoch 17/30
63/63 [==============================] - ETA: 0s - loss: 2.5014
Epoch 17: val_loss improved from 2.56513 to 2.53765, saving model to kaggle_model
63/63 [==============================] - 15s 238ms/step - loss: 2.5014 - val_loss: 2.5377
Epoch 18/30
62/63 [============================>.] - ETA: 0s - loss: 2.4828
Epoch 18: val_loss improved from 2.53765 to 2.51946, saving model to kaggle_model
63/63 [==============================] - 14s 232ms/step - loss: 2.4823 - val_loss: 2.5195
Epoch 19/30
62/63 [============================>.] - ETA: 0s - loss: 2.4610
Epoch 19: val_loss improved from 2.51946 to 2.49545, saving model to kaggle_model
63/63 [==============================] - 14s 231ms/step - loss: 2.4607 - val_loss: 2.4955
Epoch 20/30
63/63 [==============================] - ETA: 0s - loss: 2.4367
Epoch 20: val_loss improved from 2.49545 to 2.48011, saving model to kaggle_model
63/63 [==============================] - 18s 287ms/step - loss: 2.4367 - val_loss: 2.4801
Epoch 21/30
62/63 [============================>.] - ETA: 0s - loss: 2.4123
Epoch 21: val_loss improved from 2.48011 to 2.45605, saving model to kaggle_model
63/63 [==============================] - 15s 234ms/step - loss: 2.4125 - val_loss: 2.4560
Epoch 22/30
63/63 [==============================] - ETA: 0s - loss: 2.3906
Epoch 22: val_loss improved from 2.45605 to 2.44242, saving model to kaggle_model
63/63 [==============================] - 16s 250ms/step - loss: 2.3906 - val_loss: 2.4424
Epoch 23/30
63/63 [==============================] - ETA: 0s - loss: 2.3700
Epoch 23: val_loss improved from 2.44242 to 2.43009, saving model to kaggle_model
63/63 [==============================] - 15s 245ms/step - loss: 2.3700 - val_loss: 2.4301
Epoch 24/30
63/63 [==============================] - ETA: 0s - loss: 2.3485
Epoch 24: val_loss improved from 2.43009 to 2.39692, saving model to kaggle_model
63/63 [==============================] - 15s 248ms/step - loss: 2.3485 - val_loss: 2.3969
Epoch 25/30
62/63 [============================>.] - ETA: 0s - loss: 2.3292
Epoch 25: val_loss did not improve from 2.39692
63/63 [==============================] - 4s 66ms/step - loss: 2.3288 - val_loss: 2.4199
Epoch 26/30
63/63 [==============================] - ETA: 0s - loss: 2.3101
Epoch 26: val_loss improved from 2.39692 to 2.39602, saving model to kaggle_model
63/63 [==============================] - 14s 227ms/step - loss: 2.3101 - val_loss: 2.3960
Epoch 27/30
63/63 [==============================] - ETA: 0s - loss: 2.2905
Epoch 27: val_loss improved from 2.39602 to 2.34717, saving model to kaggle_model
63/63 [==============================] - 16s 258ms/step - loss: 2.2905 - val_loss: 2.3472
Epoch 28/30
62/63 [============================>.] - ETA: 0s - loss: 2.2696
Epoch 28: val_loss improved from 2.34717 to 2.33483, saving model to kaggle_model
63/63 [==============================] - 15s 241ms/step - loss: 2.2698 - val_loss: 2.3348
Epoch 29/30
62/63 [============================>.] - ETA: 0s - loss: 2.2510
Epoch 29: val_loss improved from 2.33483 to 2.31494, saving model to kaggle_model
63/63 [==============================] - 15s 243ms/step - loss: 2.2508 - val_loss: 2.3149
Epoch 30/30
62/63 [============================>.] - ETA: 0s - loss: 2.2310
Epoch 30: val_loss improved from 2.31494 to 2.28819, saving model to kaggle_model
63/63 [==============================] - 15s 242ms/step - loss: 2.2310 - val_loss: 2.2882

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.title('Bi-directional LSTM+multi-self-attention in encoder Model Loss')

plt.savefig("plot.svg", format='svg')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.title('Kaggle Model Loss')

plt.savefig("plot2.svg", format='svg')
plt.show()

In [ ]:
import numpy as np
model = load_model('/content/kaggle_model')
# Pred
preds = model.predict(testX.reshape((testX.shape[0],testX.shape[1])))
# Convert predictions to classes
preds_classes = np.argmax(preds, axis=-1)

In [ ]:
import numpy as np
model = load_model('/content/Proposed_model_5')
# Pred
preds = model.predict(testX.reshape((testX.shape[0],testX.shape[1])))
# Convert predictions to classes
preds_classes = np.argmax(preds, axis=-1)

In [ ]:
preds_classes

In [ ]:
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

In [ ]:
# Convert predictions into text (English)
preds_text = []
for i in preds_classes:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], eng_tokenizer)
        if j > 0:
            if (t == get_word(i[j-1], eng_tokenizer)) or (t == None):
                temp.append('')
            else:
                temp.append(t)
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t)
    preds_text.append(' '.join(temp))

In [ ]:
print(len(test[:,0]))
print(len(preds_text))

In [ ]:
pred_df = pd.DataFrame({'actual' : test[:,0], 'predicted' : preds_text})

In [ ]:
# print 15 rows randomly
pred_df.head(15)

In [ ]:
# print 15 rows randomly
pred_df.head(15)

In [ ]:
import shutil

# Define the path to your checkpoint directory
checkpoint_path = '/content/kaggle_model'

# Create a zip file of the checkpoints
shutil.make_archive(checkpoint_path, 'zip', checkpoint_path)

# Provide the path to the zip file
zip_file_path = checkpoint_path + '.zip'

# Move the zip file to the Colab download directory
shutil.move(zip_file_path, '/content')

# Provide the path to the zip file in the Colab download directory
download_path = '/content/' + os.path.basename(zip_file_path)

# Trigger the download of the zip file
from google.colab import files
files.download(download_path)

